In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Tishad Bhai's Project

In [ ]:
# Regular EDA(exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

from tqdm import tqdm

In [ ]:
data=pd.read_excel('/kaggle/input/civil-eng/final_excell_new.xlsx')

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.dtypes

In [ ]:
data.drop(['Date'], axis=1, inplace=True)
data.head(3)

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
# Make a corelation matrix
import seaborn as sb

corr_matrix = data.corr()
fig, ax = plt.subplots(figsize=(12, 7))

ax = sb.heatmap(corr_matrix,
                annot=True,
                linewidths=0.5,
                fmt='.2f',
                cmap='YlGnBu')

In [ ]:
corr_mat_data = data.corr()

data_mask = np.triu(np.ones_like(corr_mat_data, dtype = bool))

cmap = sb.diverging_palette(100, 7, s = 75, l = 40, n = 20, center = 'light', as_cmap = True)

fig, axes = plt.subplots(figsize = (15, 5))
sb.heatmap(corr_mat_data, annot = True, cmap = cmap, fmt = '.2f', center = 0,
            annot_kws = {'size': 20}, mask = data_mask).set_title('Correlations Among Features')

In [ ]:
data.shape, data.drop_duplicates().shape

### No duplicate Values

## Box Whisker Plots

In [ ]:
cnm = data.columns

In [ ]:
for cols in cnm:
    fig, axes = plt.subplots(figsize = (8, 2))
    sb.boxplot(data = data, x = cols).set_title(cols);

In [ ]:
cnm

### Precipitation VS all Graph

In [ ]:
for cols in cnm:
    fig, axes = plt.subplots(figsize = (6, 5))
    sb.scatterplot(data = data, x = cols,
                   y = cnm[2], color = 'steelblue').set_title('Tishad Bhais Dataset')

In [ ]:
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [ ]:
from sklearn.model_selection import train_test_split
# Set the sata
X = data.drop(['precipitation (mm)'], axis=1)
y = data['precipitation (mm)']

# Split the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the lengths of the sets
print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

In [ ]:
xgb = XGBRegressor(objective = 'reg:pseudohubererror',
                          colsample_bytree = 0.9, 
                          gamma = 0.65, 
                          learning_rate = 0.01, 
                          max_depth = 7, 
                          min_child_weight = 20, 
                          n_estimators = 1000, 
                          subsample = 0.7)
xgb.fit(X_train, y_train)

## Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

def show_scores(model):
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)
    scores = {
        'training MAE': mean_absolute_error(y_train, train_preds),
        'test MAE': mean_absolute_error(y_test, test_preds),
        
        'training MAPE': mean_absolute_percentage_error(y_train, train_preds),
        'test MAPE': mean_absolute_percentage_error(y_test, test_preds),
        
        'training MSE': mean_squared_error(y_train, train_preds),
        'test MSE': mean_squared_error(y_test, test_preds),
        
        'Training R^2': r2_score(y_train, train_preds),
        'Valid R^2': r2_score(y_test, test_preds)
    }
    return scores

In [ ]:
show_scores(xgb)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error

# Define the cross-validation strategy
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and calculate MAE
mae_scores = -cross_val_score(xgb, X, y, cv=kfold, scoring='neg_mean_absolute_error')

# Calculate the average MAE score
average_mae = np.mean(mae_scores)

print('MAE scores:', mae_scores)
print('Average MAE:', average_mae)

In [ ]:
cat = CatBoostRegressor(loss_function = 'MAE',
                               iterations = 1000,
                               learning_rate = 0.08,
                               depth = 10, 
                               random_strength = 0.2,
                               bagging_temperature = 0.7,
                               border_count = 254,
                               l2_leaf_reg = 0.001,
                               verbose = False,
                               grow_policy = 'Lossguide',
                               task_type = 'CPU',
                               random_state = 42).fit(X_train, y_train)
    

In [ ]:
show_scores(cat)